In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# create a dask client and local cluster
from dask.distributed import Client, LocalCluster
cluster = LocalCluster()
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 31.64 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:63461,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:63483,Total threads: 4
Dashboard: http://127.0.0.1:63489/status,Memory: 7.91 GiB
Nanny: tcp://127.0.0.1:63464,


In [3]:
# import a parquet file as a pandas dataframe
import dask.dataframe as dd
df = dd.read_parquet('Flight_Info_2018.parquet')

In [4]:
# see a preview of the data
df.head()

,FlightDate,Airline,Origin,Dest,CRSDepTime,DepTime,AirTime,CRSElapsedTime,ActualElapsedTime,Distance,...,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,DistanceGroup,DivAirportLandings,Diverted_,Dep_Delay
0,2018-01-18T00:00,Endeavor Air Inc.,ATL,ABY,1037,1233.0,32.0,60.0,72.0,145.0,...,7.0,1000-1059,34.0,1307.0,1339.0,6.0,1,0.0,0,delayed
1,2018-01-13T00:00,Endeavor Air Inc.,ATL,EVV,941,1142.0,54.0,87.0,83.0,350.0,...,8.0,0900-0959,22.0,1204.0,1158.0,7.0,2,0.0,0,delayed
2,2018-01-29T00:00,Endeavor Air Inc.,ATL,EVV,939,1012.0,58.0,89.0,81.0,350.0,...,2.0,0900-0959,19.0,1031.0,1029.0,4.0,2,0.0,0,delayed
3,2018-01-06T00:00,Endeavor Air Inc.,BUF,DTW,1006,1043.0,39.0,84.0,77.0,241.0,...,2.0,1000-1059,29.0,1112.0,1151.0,9.0,1,0.0,0,delayed
4,2018-01-08T00:00,Endeavor Air Inc.,BTV,DTW,1240,1254.0,92.0,135.0,117.0,537.0,...,0.0,1200-1259,15.0,1309.0,1441.0,10.0,3,0.0,0,delayed


In [5]:
df_clean = df.drop(columns=['Flight_Number_Operating_Airline', 'OriginAirportID', 'OriginWac','DestWac','DestAirportID','CRSElapsedTime','AirTime','CRSDepTime','DepTime','FlightDate','Origin','Dest', 'DepartureDelayGroups','Distance'])

In [6]:
catcolumnslist = ['Airline',
 'OriginStateName',
 'DestStateName',
 'DepTimeBlk',
 'DistanceGroup']

In [7]:
from dask_ml.preprocessing import OneHotEncoder
# Initialize and fit the encoder
encoder = OneHotEncoder()
df_clean = df_clean.categorize(columns=catcolumnslist)
encoder.fit(df_clean[catcolumnslist])

# Transform the data and persist the result
encoded_df = encoder.transform(df_clean[catcolumnslist])
result = dd.concat([df_clean.drop(catcolumnslist, axis=1), encoded_df], axis=1).persist()

In [8]:
result = result.compute()
type(result)


pandas.core.frame.DataFrame

In [9]:
client.close()
cluster.close()

In [10]:
result.head()

,ActualElapsedTime,TaxiOut,WheelsOff,WheelsOn,TaxiIn,DivAirportLandings,Diverted_,Dep_Delay,Airline_Air Wisconsin Airlines Corp,Airline_Alaska Airlines Inc.,...,DistanceGroup_2,DistanceGroup_3,DistanceGroup_4,DistanceGroup_5,DistanceGroup_6,DistanceGroup_7,DistanceGroup_8,DistanceGroup_9,DistanceGroup_10,DistanceGroup_11
0,72.0,34.0,1307.0,1339.0,6.0,0.0,0,delayed,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,83.0,22.0,1204.0,1158.0,7.0,0.0,0,delayed,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,81.0,19.0,1031.0,1029.0,4.0,0.0,0,delayed,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,77.0,29.0,1112.0,1151.0,9.0,0.0,0,delayed,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,117.0,15.0,1309.0,1441.0,10.0,0.0,0,delayed,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
result.dropna(axis=1, inplace=True)

In [12]:
# apply a decision tree model to the cleaned data
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
X = result.drop('Dep_Delay', axis=1)
y = result['Dep_Delay']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

c:\Users\Nilotpal.Choudhury\pyworkspaces\.env_py3139\Lib\site-packages\sklearn\utils\validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
c:\Users\Nilotpal.Choudhury\pyworkspaces\.env_py3139\Lib\site-packages\sklearn\utils\validation.py:919: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


              precision    recall  f1-score   support

     delayed       0.62      0.66      0.64    101163
 not delayed       0.63      0.59      0.61    100787

    accuracy                           0.62    201950
   macro avg       0.62      0.62      0.62    201950
weighted avg       0.62      0.62      0.62    201950

